In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
import torch.nn.functional as F
import numpy as np
import glob 
import os
import numpy as np
import bdpy
from PIL import Image
import torchvision.models as models
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

In [ ]:
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

In [ ]:
class Generator(nn.Module):
    def __init__(self, batch_size):
        super(Generator, self).__init__()
        self.batch_size = batch_size
        self.defc7 = nn.Linear(9919 + 9216, 4096)
        self.relu_defc7 = nn.ReLU(0.3)
        nn.init.kaiming_normal_(self.defc7.weight, mode='fan_in', nonlinearity='relu')
        nn.init.constant_(self.defc7.bias, 0)
        

        self.defc6 = nn.Linear(4096, 4096)
        self.relu_defc6 = nn.ReLU(0.3)
        nn.init.kaiming_normal_(self.defc6.weight, mode='fan_in', nonlinearity='relu')
        nn.init.constant_(self.defc6.bias, 0)

        self.defc5 = nn.Linear(4096, 4096)
        self.relu_defc5 = nn.ReLU(0.3)
        nn.init.kaiming_normal_(self.defc5.weight, mode='fan_in', nonlinearity='relu')
        nn.init.constant_(self.defc5.bias, 0)

        self.deconv5 = nn.ConvTranspose2d(256, 256, kernel_size=4, stride=2, padding=1)
        self.relu_deconv5 = nn.ReLU(0.3)
        nn.init.kaiming_normal_(self.deconv5.weight, a=1.8, mode='fan_out', nonlinearity='relu')
        nn.init.constant_(self.deconv5.bias, 0)
        
        

        self.conv5_1 = nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1)  # Convolution
        self.relu_conv5_1 = nn.ReLU(0.3)
        nn.init.kaiming_normal_(self.conv5_1.weight, a=0.9, mode='fan_out', nonlinearity='relu')
        nn.init.constant_(self.conv5_1.bias, 0)

        self.deconv4 = nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1)  # Deconvolution
        self.relu_deconv4 = nn.ReLU(0.3)
        nn.init.kaiming_normal_(self.deconv4.weight, a=1.8, mode='fan_out', nonlinearity='relu')
        nn.init.constant_(self.deconv4.bias, 0)

        self.conv4_1 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1)  # Convolution
        self.relu_conv4_1 = nn.ReLU(0.3)
        nn.init.kaiming_normal_(self.conv4_1.weight, a=0.9, mode='fan_out', nonlinearity='relu')
        nn.init.constant_(self.conv4_1.bias, 0)

        self.deconv3 = nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1)  # Deconvolution
        self.relu_deconv3 = nn.ReLU(0.3)
        nn.init.kaiming_normal_(self.deconv3.weight, a=1.8, mode='fan_out', nonlinearity='relu')
        nn.init.constant_(self.deconv3.bias, 0)

        self.conv3_1 = nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1)  # Convolution
        self.relu_conv3_1 = nn.ReLU(0.3)
        nn.init.kaiming_normal_(self.conv3_1.weight, a=0.9, mode='fan_out', nonlinearity='relu')
        nn.init.constant_(self.conv3_1.bias, 0)

        self.deconv2 = nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1)
        self.relu_deconv2 = nn.ReLU(0.3)
        nn.init.kaiming_normal_(self.deconv2.weight, a=1.8, mode='fan_out', nonlinearity='relu')
        nn.init.constant_(self.deconv2.bias, 0)

        self.deconv1 = nn.ConvTranspose2d(64, 32, kernel_size=4, stride=2, padding=1)
        self.relu_deconv1 = nn.ReLU(0.3)
        nn.init.kaiming_normal_(self.deconv1.weight, a=1.8, mode='fan_out', nonlinearity='relu')
        nn.init.constant_(self.deconv1.bias, 0)

        self.deconv0 = nn.ConvTranspose2d(32, 3, kernel_size=4, stride=2, padding=1)
        nn.init.kaiming_normal_(self.deconv0.weight, a=1.8, mode='fan_out', nonlinearity='relu')
        nn.init.constant_(self.deconv0.bias, 0)

    def forward(self, x):
        
        x = self.defc7(x)
        x = self.relu_defc7(x)
        x = self.defc6(x)
        x = self.relu_defc6(x)
        x = self.defc5(x)
        x = self.relu_defc5(x)
        x = x.reshape(x.shape[0],256,4,4)
        x = self.deconv5(x)
        x = self.relu_deconv5(x)
        x = self.conv5_1(x)
        x = self.relu_conv5_1(x)
        x = self.deconv4(x)
        x = self.relu_deconv4(x)
        x = self.conv4_1(x)
        x = self.relu_conv4_1(x)
        x = self.deconv3(x)
        x = self.relu_deconv3(x)
        x = self.conv3_1(x)
        x = self.relu_conv3_1(x)
        x = self.deconv2(x)
        x = self.relu_deconv2(x)
        x = self.deconv1(x)
        x = self.relu_deconv1(x)
        x = self.deconv0(x)
        _, _, height, width = x.size()
        start_h = (height - 227) // 2
        start_w = (width - 227) // 2
        cropped_x = torch.narrow(x, 2, start_h, 227)
        cropped_x = torch.narrow(cropped_x, 3, start_w, 227)
        
        return cropped_x

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, batch_size):
        super(Discriminator, self).__init__()
        self.batch_size = batch_size
        self.Dconv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=7, stride=4)
        nn.init.kaiming_normal_(self.Dconv1.weight, a=0.1, mode='fan_in', nonlinearity='relu')
        nn.init.constant_(self.Dconv1.bias, 0)
        self.Drelu1 = nn.ReLU()

        self.Dconv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, stride=1)
        nn.init.kaiming_normal_(self.Dconv2.weight, a=1, mode='fan_in', nonlinearity='relu')
        nn.init.constant_(self.Dconv2.bias, 0)
        self.Drelu2 = nn.ReLU()

        self.Dconv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=2)
        nn.init.kaiming_normal_(self.Dconv3.weight, a=1, mode='fan_in', nonlinearity='relu')
        nn.init.constant_(self.Dconv1.bias, 0)
        self.Drelu3 = nn.ReLU()

        self.Dconv4 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1)
        nn.init.kaiming_normal_(self.Dconv4.weight, a=1, mode='fan_in', nonlinearity='relu')
        nn.init.constant_(self.Dconv4.bias, 0)
        self.Drelu4 = nn.ReLU()

        self.Dconv5 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=2)
        nn.init.kaiming_normal_(self.Dconv5.weight, a=1, mode='fan_in', nonlinearity='relu')
        nn.init.constant_(self.Dconv5.bias, 0)
        self.relu5 = nn.ReLU()

        self.Dpool5 = nn.AvgPool2d(kernel_size=11, stride=11)
        self.drop5 = nn.Dropout(p=0.5)

        self.Dfc6 = nn.Linear(256, 256)
        nn.init.kaiming_normal_(self.Dfc6.weight, a=0.1, mode='fan_in', nonlinearity='relu')
        nn.init.constant_(self.Dfc6.bias, 0)
        self.Drelu6 = nn.ReLU()
        self.drop6 = nn.Dropout(p=0.5)

        self.Dfc7 = nn.Linear(256, 2)
        nn.init.kaiming_normal_(self.Dfc7.weight, a=0.1, mode='fan_in', nonlinearity='relu')
        nn.init.constant_(self.Dfc7.bias, 0)
    def forward(self, x):
        x = self.Dconv1(x)
        x = self.Drelu1(x)
        x = self.Dconv2(x)
        x = self.Drelu2(x)
        x = self.Dconv3(x)
        x = self.Drelu3(x)
        x = self.Dconv4(x)
        x = self.Drelu4(x)
        x = self.Dconv5(x)
        x = self.relu5(x)
        x = self.Dpool5(x)
        x = x.reshape(x.shape[0],256)
        x = self.drop5(x)
        x = self.Dfc6(x)
        x = self.Drelu6(x)
        x = self.drop6(x)
        x = self.Dfc7(x)
        return x

In [ ]:
class Comparator(nn.Module):
    def __init__(self):
        super(Comparator, self).__init__()
        alexnet = models.alexnet(pretrained=True)

        self.alexnet_without_fc = torch.nn.Sequential(*(list(alexnet.features.children())))

    def forward(self, x):   
        with torch.no_grad():
            self.alexnet_without_fc.eval()
            features = self.alexnet_without_fc(x)
            return features

In [ ]:
fmri_data_table = [
    {'subject': 'sub-01',
     'data_file': '../data/fmri/sub-01_perceptionNaturalImageTraining_original_VC.h5',
     'roi_selector': 'ROI_VC = 1'},
    {'subject': 'sub-02',
     'data_file': '../data/fmri/sub-02_perceptionNaturalImageTraining_original_VC.h5',
     'roi_selector': 'ROI_VC = 1'},
    {'subject': 'sub-03',
     'data_file': '../data/fmri/sub-03_perceptionNaturalImageTraining_original_VC.h5',
     'roi_selector': 'ROI_VC = 1'}
]

In [ ]:
fmri_data = np.load('./subj3_fmri.npy')
image_features = np.load('./image_features.npy')

In [ ]:
image_dir = '../data/images/training'
image_file_pattern = '*.JPEG'

In [ ]:
fmri_data_bd = bdpy.BData(fmri_data_table[2]['data_file'])

In [ ]:
images_list = glob.glob(os.path.join(image_dir, image_file_pattern))  # List of image files (full path)
images_table = {os.path.splitext(os.path.basename(f))[0]: f
                    for f in images_list}
label_table = {os.path.splitext(os.path.basename(f))[0]: i + 1
                   for i, f in enumerate(images_list)}  

In [ ]:
fmri_labels = fmri_data_bd.get('Label')[:, 1].flatten()
fmri_labels = ['n%08d_%d' % (int(('%f' % a).split('.')[0]),
                                 int(('%f' % a).split('.')[1]))
                   for a in fmri_labels]

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, fmri_data, image_features, frmi_labels, images_table):
        self.fmri_data = fmri_data
        self.image_features = image_features
        
        self.ridge_regressor = Ridge(alpha=1.0) 
        self.ridge_regressor.fit(fmri_data, image_features)

        self.pred_features = self.ridge_regressor.predict(fmri_data)
    def __len__(self):
        return len(fmri_data)
    def __getitem__(self, idx):
        image_path = images_table[fmri_labels[idx]]
        preprocess = transforms.Compose([
        transforms.Resize(248),
        transforms.CenterCrop(227),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])
        image = Image.open(image_path)
        image = np.asarray(image)
        if image.ndim == 2:
            img_rgb = np.zeros((image.shape[0], image.shape[1], 3), dtype=image.dtype)
            img_rgb[:, :, 0] = image
            img_rgb[:, :, 1] = image
            img_rgb[:, :, 2] = image
            image = img_rgb
        input_tensor = preprocess(Image.fromarray(image))
        return torch.Tensor(fmri_data[idx]), torch.Tensor(image_features[idx]), torch.Tensor(self.pred_features[idx]),input_tensor
traindata = CustomDataset(fmri_data, image_features, fmri_labels, images_table)

In [ ]:
generator = torch.load('./generator_no_image.pt')

In [ ]:
import numpy as np
import cv2
from scipy.stats import pearsonr
from skimage.metrics import structural_similarity as ssim

def calculate_metrics(original_image, predicted_image):
    original_image = np.transpose(original_image, (1,2,0))
    predicted_image = np.transpose(predicted_image, (1,2,0))
    corr_coefficient, _ = pearsonr(original_image.flatten(), predicted_image.flatten())
    min_values = np.min(predicted_image, axis=(0, 1))
    max_values = np.max(predicted_image, axis=(0, 1))
    predicted_image = (predicted_image - min_values)/(max_values-min_values)
    ssim_value = ssim(original_image, predicted_image,data_range=1.0, channel_axis=2)
    # ssim_value=0
    return corr_coefficient, ssim_value

def process_batch(batch_original_images, batch_predicted_images):
    batch_corr_coefficients = []
    batch_ssim_values = []
    
    for original_image, predicted_image in zip(batch_original_images, batch_predicted_images):
        corr_coefficient, ssim_value = calculate_metrics(original_image, predicted_image)
        batch_corr_coefficients.append(corr_coefficient)
        batch_ssim_values.append(ssim_value)

    
    return batch_corr_coefficients, batch_ssim_values


In [ ]:
import numpy as np
generator.eval()
batch_size = 1200
data_loader = DataLoader(traindata, batch_size=batch_size)
print(len(data_loader))
pearsons = []
ssims = []
with torch.no_grad():
    for i, (fmri_data_batch,_,pred_features,original_image) in enumerate(data_loader):
        # generated_image = generator(torch.concat((fmri_data_batch, pred_features), dim=1))
        generated_image = generator(fmri_data_batch)

        pearson, ssim_batch = process_batch(original_image.numpy(), generated_image.numpy())
        pearsons.append(np.mean(pearson))
        ssims.append(np.mean(ssim_batch))
print("Pearson Correlation: ", np.mean(pearsons))
print("SSIM: ", np.mean(ssims))



In [ ]:
generator.eval()
num_images = 50
fig, axes = plt.subplots(num_images, 2, figsize=(10, 2*num_images))
batch_size = 1200
data_loader = DataLoader(traindata, batch_size=batch_size)
with torch.no_grad():
    for i, (fmri_data_batch,_,pred_features,original_image) in enumerate(data_loader):
        generated_image = generator(torch.concat((fmri_data_batch, pred_features), dim=1))
        for j in range(100,150):
            i = j - 100
            axes[i, 0].imshow(np.transpose(original_image[i], (1, 2, 0)))
            axes[i, 0].set_title('Original Image')
            axes[i, 0].axis('off')
            generated_image_curr = np.transpose(generated_image[i], (1, 2, 0)).numpy()
            # generated_image = generated_image - generated_image.min()
            # generated_image = generated_image * (1/generated_image.max())
            # mean = np.mean(generated_image[i], axis=(0, 1))
            # std = np.std(generated_image[i], axis=(0, 1))
            min_values = np.min(generated_image_curr, axis=(0, 1))
            max_values = np.max(generated_image_curr, axis=(0, 1))
            # print(mean, std)
            axes[i, 1].imshow((generated_image_curr - min_values)/(max_values-min_values))
            axes[i, 1].set_title('Generated Image')
            axes[i, 1].axis('off')
        break
plt.tight_layout()
plt.show()



In [ ]:
print(generated_image[2])

In [ ]:
print(traindata[0][2])

In [ ]:
fmri_data_table_test = [
    {'subject': 'sub-01',
     'data_file': '../data/fmri/sub-01_perceptionNaturalImageTraining_original_VC.h5',
     'roi_selector': 'ROI_VC = 1'},
    {'subject': 'sub-02',
     'data_file': '../data/fmri/sub-02_perceptionNaturalImageTraining_original_VC.h5',
     'roi_selector': 'ROI_VC = 1'},
    {'subject': 'sub-03',
     'data_file': '../data/fmri/sub-03_perceptionNaturalImageTest_original_VC.h5',
     'roi_selector': 'ROI_VC = 1'}
]

In [ ]:
fmri_data_bd = bdpy.BData(fmri_data_table_test[2]['data_file'])

In [ ]:
image_dir_test = '../data/images/test'
image_file_pattern = '*.JPEG'

In [ ]:
images_list_test = glob.glob(os.path.join(image_dir_test, image_file_pattern))  # List of image files (full path)
images_table_test = {os.path.splitext(os.path.basename(f))[0]: f
                    for f in images_list_test}
label_table_test = {os.path.splitext(os.path.basename(f))[0]: i + 1
                   for i, f in enumerate(images_list_test)}  

In [ ]:
fmri_labels_test = fmri_data_bd.get('Label')[:, 1].flatten()
fmri_labels_test = ['n%08d_%d' % (int(('%f' % a).split('.')[0]),
                                 int(('%f' % a).split('.')[1]))
                   for a in fmri_labels_test]

In [ ]:
fmri_data_test = np.load('subj3_fmri_test.npy')

In [ ]:
class CustomDatasetTest(Dataset):
    def __init__(self, fmri_data,fmri_labels, images_table):
        self.fmri_data = fmri_data
        self.fmri_labels = fmri_labels
        self.images_table = images_table
        self.pred_features = traindata.ridge_regressor.predict(fmri_data)
    def __len__(self):
        return len(self.fmri_data)
    def __getitem__(self, idx):
        image_path = self.images_table[self.fmri_labels[idx]]
        preprocess = transforms.Compose([
        transforms.Resize(248),
        transforms.CenterCrop(227),
        transforms.ToTensor(),
        # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])
        image = Image.open(image_path)
        image = np.asarray(image)
        if image.ndim == 2:
            img_rgb = np.zeros((image.shape[0], image.shape[1], 3), dtype=image.dtype)
            img_rgb[:, :, 0] = image
            img_rgb[:, :, 1] = image
            img_rgb[:, :, 2] = image
            image = img_rgb
        input_tensor = preprocess(Image.fromarray(image))
        return torch.Tensor(self.fmri_data[idx]),torch.Tensor(self.pred_features[idx]),input_tensor
testdata = CustomDatasetTest(fmri_data_test,fmri_labels_test, images_table_test)

In [ ]:
print(fmri_data.shape)

In [ ]:
generator.eval()
batch_size = 1200
data_loader = DataLoader(testdata, batch_size=batch_size)
pearsons = []
ssims = []
with torch.no_grad():
    for i, (fmri_data,pred_features,original_image) in enumerate(data_loader):
        generated_image = generator(torch.concat((fmri_data,pred_features), dim=1))
        pearson, ssim_batch = process_batch(original_image.numpy(), generated_image.numpy())
        pearsons.append(np.mean(pearson))
        ssims.append(np.mean(ssim_batch))
print("Pearson Correlation: ", np.mean(pearsons))
print("SSIM: ", np.mean(ssims))


In [ ]:
generator.eval()
num_images = 50
fig, axes = plt.subplots(num_images, 2, figsize=(10, 2*num_images))
batch_size = 1200
data_loader = DataLoader(testdata, batch_size=batch_size)
with torch.no_grad():
    for i, (fmri_data,pred_features,original_image) in enumerate(data_loader):
        generated_image = generator(torch.concat((fmri_data, pred_features), dim=1))
        for j in range(100,150):
            i = j - 100
            axes[i, 0].imshow(np.transpose(original_image[i], (1, 2, 0)))
            axes[i, 0].set_title('Original Image')
            axes[i, 0].axis('off')
            generated_image_curr = np.transpose(generated_image[i], (1, 2, 0)).numpy()
            # generated_image = generated_image - generated_image.min()
            # generated_image = generated_image * (1/generated_image.max())
            # mean = np.mean(generated_image[i], axis=(0, 1))
            # std = np.std(generated_image[i], axis=(0, 1))
            min_values = np.min(generated_image_curr, axis=(0, 1))
            max_values = np.max(generated_image_curr, axis=(0, 1))
            
            # print(mean, std)
            pct = 0.04
            generated_image_curr[:,:,0] = np.clip(generated_image_curr[:,:,0], np.percentile(generated_image_curr[:,:,0], pct/2.),
                  np.percentile(generated_image_curr[:,:,0], 100-pct/2.))
            generated_image_curr[:,:,1] = np.clip(generated_image_curr[:,:,1], np.percentile(generated_image_curr[:,:,1], pct/2.),
                  np.percentile(generated_image_curr[:,:,1], 100-pct/2.))
            generated_image_curr[:,:,2] = np.clip(generated_image_curr[:,:,2], np.percentile(generated_image_curr[:,:,2], pct/2.),
                  np.percentile(generated_image_curr[:,:,2], 100-pct/2.))
            generated_image_curr = (generated_image_curr - min_values)/(max_values-min_values)
            axes[i, 1].imshow((generated_image_curr))
            axes[i, 1].set_title('Generated Image')
            axes[i, 1].axis('off')
plt.tight_layout()
plt.show()



In [ ]:
data_table = [
    # {
    #     'subject': 'sub-01',

    #     # The file of the test fmri data
    #     'data_fmri_test': './data/fmri/sub-01_perceptionNaturalImageTest_original_VC.h5',

    #     # The file of the training fmri data
    #     'data_fmri_training': './data/fmri/sub-01_perceptionNaturalImageTraining_original_VC.h5',

    #     # Path to caffemodel trained generator (caffemodel and prototxt files).
    #     # 'generator_caffemodel' can be either path to a caffemodel file or
    #     # a directory that contains snapshots of trained models.
    #     'generator_caffemodel': './net_pretrained/sub-01/generator.caffemodel',
    #     'generator_prototxt': './net_pretrained/sub-01/generator_test.prototxt',
    #     # 'generator_caffemodel': './net_trained/sub-01/snapshots',
    #     # 'generator_prototxt': './net_trained/sub-01/net/generator_test.prototxt',

    #     # Data select expression specifying columns (ROIs) used as the test and training data
    #     'roi_selector': 'ROI_VC = 1',

    #     # Output directory
    #     'output_dir': os.path.join(output_dir_base, 'sub-01')
    # },
    # {
    #     'subject': 'sub-02',
    #     'data_fmri_test': './data/fmri/sub-02_perceptionNaturalImageTest_original_VC.h5',
    #     'data_fmri_training': './data/fmri/sub-02_perceptionNaturalImageTraining_original_VC.h5',
    #     'generator_caffemodel': './net_pretrained/sub-02/generator.caffemodel',
    #     'generator_prototxt': './net_pretrained/sub-02/generator_test.prototxt',
    #     # 'generator_caffemodel': './net_trained/sub-02/snapshots',
    #     # 'generator_prototxt': './net_trained/sub-02/net/generator_test.prototxt',
    #     'roi_selector': 'ROI_VC = 1',
    #     'output_dir': os.path.join(output_dir_base, 'sub-02')
    # },
    {
        'subject': 'sub-03',
        'data_fmri_test': '../data/fmri/sub-03_perceptionNaturalImageTest_original_VC.h5',
        'data_fmri_training': '../data/fmri/sub-03_perceptionNaturalImageTraining_original_VC.h5',
        'generator_caffemodel': './trainedmodel_sub-03/generator.caffemodel',
        'generator_prototxt': './trainedmodel_sub-03/generator_test.prototxt',
        # 'generator_caffemodel': './net_trained/sub-03/snapshots',
        # 'generator_prototxt': './net_trained/sub-03/net/generator_test.prototxt',
        'roi_selector': 'ROI_VC=1',
        # 'output_dir': os.path.join(output_dir_base, 'sub-03')
    },
]

In [ ]:
dat = data_table[0]
fmri_data_test_file = dat['data_fmri_test']
fmri_data_selector = dat['roi_selector']
fmri_data_training_file = dat['data_fmri_training']
fmri_data = bdpy.BData(fmri_data_test_file)
brain_data = fmri_data.select(fmri_data_selector)
brain_labels = fmri_data.select('Label = 1')[:, 1]


fmri_data_train = bdpy.BData(fmri_data_training_file)
brain_data_train = fmri_data_train.select(fmri_data_selector)


brain_data_mean = np.mean(brain_data_train, axis=0)
brain_data_norm = np.std(brain_data_train, axis=0)

del(fmri_data_train)
del(brain_data_train)

# Average fMRI data across trials
brain_data_ave = []
image_list = []

labels_set = np.unique(brain_labels)
for lb in labels_set:
    sample_index = brain_labels == lb
    brain_data_sample = brain_data[sample_index, :]
    brain_data_ave.append(np.mean(brain_data_sample, axis=0))

    # Convert stimulus ID to stimulus file name
    image_name = 'n%08d_%d' % (int(('%f' % lb).split('.')[0]),
                                int(('%f' % lb).split('.')[1]))
    image_list.append(image_name)

brain_data_ave = np.vstack(brain_data_ave)

# Normalize fMRI data
brain_data_ave = (brain_data_ave - brain_data_mean) / brain_data_norm

In [ ]:
scale = 4.9

In [ ]:
for i, img in enumerate(image_list):
    input_data = brain_data_ave[i, :] * scale

In [ ]:
def calculate_metrics2(original_image, predicted_image):
    original_image = np.transpose(original_image.numpy(), (1,2,0))
    corr_coefficient, _ = pearsonr(original_image.flatten(), predicted_image.flatten())
    ssim_value = ssim(original_image, predicted_image,data_range=1.0, channel_axis=2)
    # ssim_value=0
    return corr_coefficient, ssim_value

In [ ]:
import imageio


In [ ]:
generator.eval()
num_images = 50
pearsons = []
ssims = []
fig, axes = plt.subplots(num_images, 2, figsize=(10, 2*num_images))
with torch.no_grad():
    for i, image_path in enumerate(image_list):
        name = image_path
        image_path = '../data/images/test/' + image_path + '.JPEG'
        preprocess = transforms.Compose([
        transforms.Resize(248),
        transforms.CenterCrop(227),
        transforms.ToTensor(),
        # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])
        image = Image.open(image_path)
        image = np.asarray(image)
        if image.ndim == 2:
            img_rgb = np.zeros((image.shape[0], image.shape[1], 3), dtype=image.dtype)
            img_rgb[:, :, 0] = image
            img_rgb[:, :, 1] = image
            img_rgb[:, :, 2] = image
            image = img_rgb
        original_image = preprocess(Image.fromarray(image))
        input_data = brain_data_ave[i, :] * scale
        pred_features = traindata.ridge_regressor.predict(input_data.reshape(1,-1))
        generated_image = generator(torch.concat((torch.tensor(input_data, dtype=torch.float).unsqueeze(0), torch.tensor(pred_features, dtype=torch.float)), dim = 1))
        axes[i, 0].imshow(np.transpose(original_image, (1, 2, 0)))
        axes[i, 0].set_title('Original Image')
        axes[i, 0].axis('off')
        generated_image_curr = np.transpose(generated_image[0], (1, 2, 0)).numpy()
        # generated_image = generated_image - generated_image.min()
        # generated_image = generated_image * (1/generated_image.max())
        # mean = np.mean(generated_image[i], axis=(0, 1))
        # std = np.std(generated_image[i], axis=(0, 1))
        min_values = np.min(generated_image_curr, axis=(0, 1))
        max_values = np.max(generated_image_curr, axis=(0, 1))
        
        # print(mean, std)
        pct = 0.04
        generated_image_curr[:,:,0] = np.clip(generated_image_curr[:,:,0], np.percentile(generated_image_curr[:,:,0], pct/2.),
                np.percentile(generated_image_curr[:,:,0], 100-pct/2.))
        generated_image_curr[:,:,1] = np.clip(generated_image_curr[:,:,1], np.percentile(generated_image_curr[:,:,1], pct/2.),
                np.percentile(generated_image_curr[:,:,1], 100-pct/2.))
        generated_image_curr[:,:,2] = np.clip(generated_image_curr[:,:,2], np.percentile(generated_image_curr[:,:,2], pct/2.),
                np.percentile(generated_image_curr[:,:,2], 100-pct/2.))
        generated_image_curr = (generated_image_curr - min_values)/(max_values-min_values)
        axes[i, 1].imshow((generated_image_curr))
        axes[i, 1].set_title('Generated Image')
        axes[i, 1].axis('off')
        curr_pearson, curr_ssim = calculate_metrics2(original_image, generated_image_curr)
        img = Image.fromarray((generated_image_curr *255).astype(np.uint8), "RGB")
        # img.save(f'./results/baseline/{name}.jpeg')
        pearsons.append(curr_pearson)
        ssims.append(curr_ssim)
print("Pearson Correlation: ", np.mean(pearsons))
print("SSIM: ", np.mean(ssims))
plt.tight_layout()
plt.show()


In [ ]:
print(pearsons)

In [ ]:
def calculate_metrics3(original_image, predicted_image):
    corr_coefficient, _ = pearsonr(original_image.flatten(), predicted_image.flatten())
    ssim_value = ssim(original_image, predicted_image,data_range=1.0, channel_axis=2)
    # ssim_value=0
    return corr_coefficient, ssim_value

In [ ]:
import os
from PIL import Image

def load_images_from_folder(folder_path):
    pearsons = []
    ssims =  []
    for filename in os.listdir(folder_path):
        
        image_name = filename.split(".")[0][7:]
        img = Image.open("./results/" + filename)
        predicted = np.asarray(img)
        img = Image.open("../data/images/test/" + image_name + ".JPEG").resize((227,227))
        original = np.asarray(img)
        pearson_curr, ssim_curr = calculate_metrics3(original, predicted)
        pearsons.append(pearson_curr)
        ssims.append(ssim_curr)
        plt.imshow(original)
        plt.imshow(predicted)
    print("Pearson Correlation: ", np.mean(pearsons))
    print("SSIM: ", np.mean(ssims))
folder_path = "./results/"
images = load_images_from_folder(folder_path)
